## 10 人脸识别（ML）

Haar特征+Adaboost分类器


#### 1. Haar特征
haar由多个模板组成

- 特征运算量大<br>
    - haar模板 从上到下 从左到右移动<br>
img-100\*100 模板-10\*10 step=10 需要100次<br>
模板还需要缩放 10\*10 -> 11\*11（每次缩放10%） 缩放20级
    - $i.e:$ 1080\*720、10\*10、step=2 <br>
计算量 = 14个模板\*20级缩放\*( 1080/2\*720/2 )( 大约是这么多保持数量级不变即可 )\*(100个点的加减运算) = 50~100亿次<br>
    且帧率一般 >=15fps，则50～100亿\*15 = 1k亿次<br>
运算量非常大. 国外提出了积分图减少运算量<br>
- 积分图，可以进行特征的快速计算<br>
每个小方块分别用1234表示<br>
A（1）B（1、2，指的是图上一行）
C（1、3，一列）
D（1234，四个方块之和）
<img src='pic_Markdown/integralMap.png' width="60%">
<br />

    - 计算方法：4 = A-B-C+D = 1 +1+2+3+4 -1-2 -1-3 =4<br>
任意一个方框可以用相邻的ABCD算出，可以把10\*10这样100个像素点的计算简化为3次加减<br>
问题是在正式计算之前，要把图片上所有像素遍历一次，计算出所有小方块
    - 训练终止条件：1）循环次数for count 2）最小检测概率p
    


#### 2. Adaboost分类器
三级（强分类器 弱分类器 node）<br>
<img src='pic_Markdown/Adaboost.png' width="60%">
一般有15-20个强分类器，只有目标都通过了这15-20个强分类器，才被认为是目标。<br>
$i.e:$ 3个强分类器各自有特征和阈值——x1 t1 x2 t2 x3 t3 <br>
if x1>t1 & x2>t2 & x4>t3， then 目标=苹果
- 强分类器=多个弱分类器，弱分类器=多个Node节点
- 弱分类器：用于计算x1、x2、x3，y是弱分类器的特征<br>
$i.e:$ 强分类器2由三个弱分类器组成，则x2 = sum(y1,y2,y3)
- Node: 用于计算y，z是node的特征，3个haar 对应 3个node节点<br>
$i.e:$ 对Node1来说，if haar1 > nodeT1，then z1=$\alpha_1$，else z1=$\alpha_2$<br>
if Z = sum(z1,z2,z3) > T ，then y1 = AA，else y1= BB

[CSDN博主讲解Adaboost](https://blog.csdn.net/v_july_v/article/details/40718799)



In [3]:
# 1 load xml 2 load img 3 haar(gray) 4 detect 5 draw rect
import cv2
import numpy as np

face_xml = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_xml = cv2.CascadeClassifier('haarcascade_eye.xml')
img = cv2.imread('face4.jpeg')
cv2.imshow('src', img)
# haar gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# detect  1 data 2 scale 3 target_size 目标最小不能小于5
faces = face_xml.detectMultiScale(gray, 1.3, 5)
print('face=',len(faces))

# draw 
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_face=gray[y:y+h,x:x+w] # gray
    roi_color = img[y:y+h,x:x+w] # BGR
    # 参数必须是gray类型
    eyes=eye_xml.detectMultiScale(roi_face)
    print('eye=',len(eyes))
    for (e_x,e_y,e_w,e_h) in eyes:
        cv2.rectangle(roi_color,(e_x,e_y),(e_x+e_w,e_y+e_h),(0,255,0),2)
cv2.imshow('dst',img)
cv2.waitKey(0)
cv2.destroyAllWindows()



face= 2
eye= 0
eye= 1
